In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions
using MacroTools

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1278


In [2]:
@model function smoothing(n, x0, c::ConstVariable, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    
    x_prev = x_prior
    
    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [3]:
P = 1.0

n = 500
data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [4]:
function inference(data, x0, P)
    n = length(data)
    
    _, (x, y) = smoothing(n, x0, 1.0, P);

    x_buffer  = buffer(Marginal, n)
    marginals = getmarginals(x)
    
    subscription = subscribe!(marginals, x_buffer)
    
    update!(y, data)
    
    unsubscribe!(subscription)
    
    return getvalues(x_buffer)
end

inference (generic function with 1 method)

In [13]:
x0_prior = NormalMeanVariance(0.0, 10000.0)
@benchmark res = inference($data, $x0_prior, $P)

BenchmarkTools.Trial: 
  memory estimate:  17.47 MiB
  allocs estimate:  294268
  --------------
  minimum time:     22.250 ms (0.00% GC)
  median time:      23.950 ms (0.00% GC)
  mean time:        27.141 ms (13.63% GC)
  maximum time:     40.148 ms (36.07% GC)
  --------------
  samples:          185
  evals/sample:     1